In [23]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# # Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-exec"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging

from deriva_ml import DatasetBag, Workflow, ExecutionConfiguration, DatasetVersion
from deriva_ml import MLVocab as vc
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [25]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2025-09-22 14:30:58,706 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-09-22 14:30:58,707 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


You are already logged in.


In [26]:
cache_dir = '/data'
working_dir = '/data'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2025-09-22 14:30:59,627 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-09-22 14:30:59,628 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


In [27]:
datasets = [
    '4-Z6K8', # New LAC balanced test set
    ]

to_be_download = []
for dataset in datasets:
    ds_dict = {
        'rid': dataset,
        'materialize':True,
        'version':EA.dataset_version(dataset_rid=dataset),
    }
    to_be_download.append(ds_dict)

workflow_instance = EA.create_workflow(
    name="Cropping Image",
    workflow_type="Image Cropping"
)

config = ExecutionConfiguration(
    datasets=to_be_download,
    assets = ['2-4JR6',],
    workflow=workflow_instance,
    description="Instance of cropping multimodal images.")

execution = EA.create_execution(config)

2025-09-22 14:31:01,617 - WARNING - File /home/kb_766/Repos/eye-ai-exec/notebooks/Sandbox_KB/HUY_crop_multimodal_images_2-N93J.ipynb has been modified since last commit. Consider commiting before executing


Execution RID: https://www.eye-ai.org/id/eye-ai/5-51CA@33Y-TM97-FBEY

2025-09-22 14:31:01,810 - INFO - Materialize bag 4-Z6K8... 
2025-09-22 14:31:02,122 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-09-22 14:31:02,122 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2025-09-22 14:31:03,844 - INFO - Using cached bag for  4-Z6K8 Version:0.5.2
2025-09-22 14:31:03,845 - INFO - Loading /data/4-Z6K8_49330658558098c69ec955ecae8b4c1a3f6c467acc788fc80427df570333a653/Dataset_4-Z6K8
2025-09-22 14:31:04,354 - INFO - Creating new database for dataset: 4-Z6K8 in /data/kb_766/EyeAI_working/4-Z6K8@33X-PGTB-JB7R.db
2025-09-22 14:31:04,402 - INFO - Downloading assets ...
2025-09-22 14:31:08,356 - INFO - File [/data/kb_766/EyeAI_working/deriva-ml/execution/5-51CA/downloaded-assets/Execution_Asset/optic_disk_crop_model.hdf5] transfer successful. 76.75 MB transferred at 2

In [28]:
print(execution)

caching_dir: /data
_working_dir: /data/kb_766/EyeAI_working
execution_rid: 5-51CA
workflow_rid: 5-515W
asset_paths: {'Execution_Asset': [AssetFilePath('/data/kb_766/EyeAI_working/deriva-ml/execution/5-51CA/downloaded-assets/Execution_Asset/optic_disk_crop_model.hdf5')]}
configuration: datasets=[DatasetSpec(rid='4-Z6K8', materialize=True, version=DatasetVersion(major=0, minor=5, patch=2))] assets=['2-4JR6'] workflow=Workflow(name='Cropping Image', url='https://github.com/informatics-isi-edu/eye-ai-exec/blob/fd01faedf2c3c685e542653216eaddb888de65ac--/notebooks/Sandbox_KB/HUY_crop_multimodal_images_2-N93J.ipynb', workflow_type='Image Cropping', version=None, description='', rid=None, checksum='b6703c771197011829f71dcce7af7028758d462f', is_notebook=True) parameters={} description='Instance of cropping multimodal images.' argv=['/home/kb_766/.conda/envs/my-tensorflow-conda/lib/python3.10/site-packages/ipykernel_launcher.py', '-f', '/home/kb_766/.local/share/jupyter/runtime/kernel-ed5f8e03-5

In [29]:
execution._working_dir

PosixPath('/data/kb_766/EyeAI_working')

In [30]:
ds_bag = execution.datasets[0]
crop_image_model =  execution.asset_paths["Execution_Asset"][0]

In [31]:
df = ds_bag.get_table_as_dataframe("Image")

In [32]:
df.rename(columns={'RID': 'RID_Image'}, inplace=True)

In [13]:
from pathlib import Path
working_dir = Path("~/working_dir")
csv_path = execution._working_dir / "df.csv"
df.to_csv(csv_path, index=False)

In [33]:
table_name = 'Image'
EA.find_features(table_name)

[Feature(target_table=Image, feature_name=Annotation, feature_table=Annotation),
 Feature(target_table=Image, feature_name=Image_Diagnosis, feature_table=Image_Diagnosis),
 Feature(target_table=Image, feature_name=Fundus_Laterality, feature_table=Execution_Image_Fundus_Laterality),
 Feature(target_table=Image, feature_name=Fundus_Angle, feature_table=Execution_Image_Fundus_Angle)]

In [34]:
ImageAnnotationFeature = EA.feature_record_class("Image", "Annotation")

In [16]:
output = execution._working_dir / "image_svg_output"
output.mkdir(parents= True, exist_ok=True)

In [ ]:

from models.vgg19.vgg19_disk_crop_predict import preprocess_and_crop
with execution.execute() as exec:
    preprocess_and_crop(
      ds_bag,
      csv_path,
      execution._working_dir / 'output_train.csv',
      'template.jpg',
      str(output),
      crop_image_model,
      "5-515T",
      "Optic Nerve OG LAC Crop",
      False
      )
    

2025-09-22 11:44:29.565274: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-22 11:44:29.565344: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-22 11:44:29.690901: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-22 11:44:29.989728: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-22 11:44:31.698080: W tensorflow/compiler/tf2

1/1 [==============================] - 1s 1s/step


2025-09-22 11:44:37,550 - INFO - SVG for 2-CXEE saved.
2025-09-22 11:44:37,668 - INFO - Processing image /data/4-Z6K8_49330658558098c69ec955ecae8b4c1a3f6c467acc788fc80427df570333a653/Dataset_4-Z6K8/data/asset/Image/1008682_1451992_14434212_Left Field 2.jpg : , trial 1, color channel grey, resize function imgResize_primary


1/1 [==============================] - 0s 21ms/step


2025-09-22 11:44:38,636 - INFO - SVG for 2-CXEJ saved.
2025-09-22 11:44:38,754 - INFO - Processing image /data/4-Z6K8_49330658558098c69ec955ecae8b4c1a3f6c467acc788fc80427df570333a653/Dataset_4-Z6K8/data/asset/Image/1008828_1452223_14435798_Right Field 2.jpg : , trial 1, color channel grey, resize function imgResize_primary


...

1/1 [==============================] - 0s 20ms/step


2025-09-22 11:56:00,716 - INFO - Processing image /data/4-Z6K8_49330658558098c69ec955ecae8b4c1a3f6c467acc788fc80427df570333a653/Dataset_4-Z6K8/data/asset/Image/1063629_1536663_15005758_Left Field 2.jpg : , trial 2, color channel green, resize function imgResize_primary


1/1 [==============================] - 0s 21ms/step


2025-09-22 11:56:01,307 - INFO - Processing image /data/4-Z6K8_49330658558098c69ec955ecae8b4c1a3f6c467acc788fc80427df570333a653/Dataset_4-Z6K8/data/asset/Image/1063629_1536663_15005758_Left Field 2.jpg : , trial 3, color channel red, resize function imgResize_primary


1/1 [==============================] - ETA: 0s

In [35]:
angle_2_only = EA.filter_angle_2(ds_bag)

In [36]:
csv_output  = Path(execution._working_dir / 'output_train.csv')

In [37]:
df_output = pd.read_csv(csv_output)

In [39]:
filtered_resulted_df = df_output[df_output["Image RID"].isin(angle_2_only["RID"])]

In [40]:
cropping_func_map_output = dict(zip(filtered_resulted_df["Image RID"], filtered_resulted_df["Worked Image Cropping Function"]))

In [41]:
import shutil 

saved_svg_names = filtered_resulted_df["Saved SVG Name"].values

bounding_box_file_paths = []
for file_name in os.listdir(output):
    if file_name in saved_svg_names:
        full_path_name = os.path.join(output, file_name)
        
        bounding_box_file = execution.asset_file_path(
            "Fundus_Bounding_Box", full_path_name, 
        )
        bounding_box_file_paths.append(bounding_box_file)

In [42]:
image_rids = []
angle_2_rids = filtered_resulted_df["Image RID"].values

for file_name in os.listdir(output):
    rid = file_name.split("_")[1].split('.')[0]
    if rid in angle_2_rids:
        image_rids.append(rid)
len(image_rids)

656

In [43]:
image_bounding_box_feature_list = []

for image_rid, asset_file_path in zip(image_rids, bounding_box_file_paths):
    expected_name = f"Cropped_{image_rid}.svg"
    actual_name = os.path.basename(asset_file_path)
    assert expected_name == actual_name, f"Expected {expected_name}, got {actual_name}"

    annotation_func = cropping_func_map_output.get(image_rid)
    if annotation_func:
        image_bounding_box_feature_list.append(
            ImageAnnotationFeature(
                Image=image_rid,
                Execution=execution.execution_rid,
                Fundus_Bounding_Box=asset_file_path,
                Annotation_Function=annotation_func,
                Annotation_Type='Optic Nerve OG LAC Crop',
            )
        )


In [45]:
with execution.execute() as execution:
    execution.add_features(image_bounding_box_feature_list)

2025-09-22 14:36:57,652 - INFO - Start execution  ...
2025-09-22 14:36:57,773 - INFO - Start execution  ...
2025-09-22 14:36:57,839 - INFO - Successfully run Ml.
2025-09-22 14:36:57,880 - INFO - Algorithm execution ended.


In [ ]:
execution.upload_execution_outputs()

2025-09-22 14:37:01,133 - INFO - Uploading execution files...
2025-09-22 14:37:01,174 - INFO - Initializing uploader: GenericUploader v1.7.7 [Python 3.10.13, Linux-5.10.210-201.852.amzn2.x86_64-x86_64-with-glibc2.26]
2025-09-22 14:37:01,175 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-09-22 14:37:01,176 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2025-09-22 14:37:01,217 - INFO - Scanning files in directory [/data/kb_766/EyeAI_working/deriva-ml/execution/5-51CA/asset]...
2025-09-22 14:37:01,240 - INFO - Including file: [/data/kb_766/EyeAI_working/deriva-ml/execution/5-51CA/asset/deriva-ml/Execution_Metadata/configuration.json].
2025-09-22 14:37:01,241 - INFO - Including file: [/data/kb_766/EyeAI_working/deriva-ml/execution/5-51CA/asset/deriva-ml/Execution_Metadata/environment_sn